## Prepare
##### Terms

- $J_i$: job $i$
- $P_l^i$: operation $l$ of job $i$
- $G_g^i$: operation group $g$ of job $i$
- $M_k$: machine $k$
- $MG_g^i$: set of machines indices that $G_g^i$ might use

##### Decision Variables

- $X_{ilj}^{k}$ := $P_l^i$ (operation $l$ of job $i$) is on the order-position $j$ on $M_k$
- $h_j^k$ := starting time of order-position $j$ on $M_k$
- $s_j^k$ := time elapsed between order-position $j$ and $j+1$ on $M_k$
- $s_0^k$ := idle time from start to first order-position on $M_k$
- $n(k)$ := max number of order-positions $M_k$ can possibly process
- $N(k)$ := job & operation pair $(i, l)$, or $(J_i, P_l^i)$, that can possibly use $M_k$
- $TX_j^k$ := process time of the $j$-th order-position task on $M_k$

##### Known Variables

- $t_{il}^{k}$ := time needed for $P_l^i$ (operation $l$ of job $i$) on using $M_k$
- $c_{il}^{S}$ := total numbers of time $P_l^i$ will use any machine $M_k \in S$ 

## Objective

\begin{align*}
\min \quad h ^ *
\end{align*}

## Constraint

##### Constraint Set 1: each job & operation is completed by its required machine(s)

\begin{align*}
&   \quad \sum_{k \in S} \sum_{j=1}^{n(k)} X_{ilj}^k = c_{il}^{S}, \ \forall (i, l)\\
\end{align*}

##### Constraint Set 2: each machine's each order-position can have at most 1 job a time

\begin{align*}
&   \quad \sum_{(i, l) \in N(k)} X_{ilj}^k \leq 1, \ \forall j \in [n(k)], \ \forall k \in M\\
\end{align*}

##### Constraint Set 3: define time variables (for ordering constraints) 

\begin{align*}
&   \quad TX_j^k = \sum_{(i, l) \in N(k)} t_{il}^k X_{ilj}^k, \ \forall j \in [n(k)], \ \forall k \in M\\
\end{align*}

\begin{align*}
&   \quad h_1^k = s_0^k, \ \forall k \in M\\
&   \quad h_j^k = \sum_{q = 1}^{j-1} TX_{q}^k + \sum_{q = 0}^{j-1} S_{q}^k, \ \forall j \in 2 \cdots n(k), \ \forall k \in M\\
\end{align*}

##### Constraint Set 4: for each job, each group must be complete in the given group order

\begin{align*}
&   \quad h_{j'}^a + t_{il'}^a X_{il'j'}^a \leq h_{j''}^b + M(1-X_{il'j'}^a) + M(1-X_{il''j''}^b) \\
&   \quad \quad \quad \forall a \in MG_g^i, \forall b \in MG_{g+1}^i, \forall P_{l'}^i \in G_g^i, \forall P_{l''}^i \in G_{g+1}^i, 
\forall j' \in [n(a)], \forall j'' \in [n(b)], \forall G_g^i \in i, \forall i
\end{align*}

##### Constraint Set 5: for each job, within each group, order doesn't matter, but must only take one machine per time

\begin{align*}
&   \quad h_{j'}^a + t_{il'}^a X_{il'j'}^a \leq h_{j''}^b + M(1-X_{il'j'}^a) + M(1-X_{il''j''}^b) + \delta M \\
&   \quad h_{j''}^a + t_{il''}^a X_{il''j''}^a \leq h_{j'}^b + M(1-X_{il'j'}^a) + M(1-X_{il''j''}^b) + (1 - \delta) M \\
&   \quad \quad \quad 
\forall (a, b) \in MG_g^i, \forall (P_{l'}^i, P_{l''}^i) \in G_g^i, 
\forall j' \in [n(a)], \forall j'' \in [n(b)], \forall G_g^i \in i, \forall i
\end{align*}

##### Constraint Set 6: define make span

\begin{align*}
&   \quad h ^ * \geq h_{n(k)}^k + TX_{n(k)}^k, \ \forall k \in M\\
\end{align*}

In [4]:
# Define your sets based on the problem
# Replace these with actual numbers
II = 3 # Number of jobs
LL = Dict(1 => 5, 2 => 6, 3 => 4) # Number of operations for each job
GG = Dict(1 => 4, 2 => 5, 3 => 3) # Number of groups for each job
MM = 6 # Number of machines
# from all (i, g) job-i group-g to machines that this group might use
MG = Dict(
    (1, 1) => [1], (1, 2) => [2], (1, 3) => [3], (1, 4) => [5, 6],
    (2, 1) => [1], (2, 2) => [2, 3], (2, 3) => [4], (2, 4) => [5, 6], (2, 5) => [5, 6],
    (3, 1) => [1, 6], (3, 2) => [6], (3, 3) => [5, 6]
) 
# max number of order-positions machine-k can possibly process
nk = Dict(1 => 3, 2 => 2, 3 => 2, 4 => 1, 5 => 2, 6 => 3)
# all (i, l) job-i operation-l pairs that can possibly use machine-k
Nk = Dict(
    1 => [(1, 1), (2, 1), (3, 1)], 
    2 => [(1, 2), (2, 2)], 
    3 => [(1, 3), (2, 3)],
    4 => [(2, 4)], 
    5 => [(1, 4), (1, 5), (2, 5), (2, 6), (3, 4)], 
    6 => [(1, 4), (1, 5), (2, 5), (2, 6), (3, 4), (3, 2), (3, 3)]
)
# GroupOps: from each (i, g) to the operation indices (l) 
GroupOps = Dict(
    (1, 1) => [1], (1, 2) => [2, 3], (1, 3) => [4], (1, 4) => [5],
    (2, 1) => [1], (2, 2) => [2, 3], (2, 3) => [4], (2, 4) => [5], (2, 5) => [6],
    (3, 1) => [1, 2], (3, 2) => [3], (3, 3) => [4]
)
# time needed for job-i operation-l to use machine-k
TILK = Dict(
    (1, 1, 1) => 60, 
    (1, 2, 2) => 30,
    (1, 3, 3) => 60,
    (1, 4, 5) => 20,
    (1, 4, 6) => 10,
    (1, 5, 5) => 5,
    (1, 5, 6) => 5,
    (2, 1, 1) => 60, 
    (2, 2, 2) => 30,
    (2, 3, 3) => 60,
    (2, 4, 4) => 20,
    (2, 5, 5) => 40,
    (2, 5, 6) => 20,
    (2, 6, 5) => 5,
    (2, 6, 6) => 5,
    (3, 1, 1) => 60, 
    (3, 2, 6) => 30,
    (3, 3, 6) => 60,
    (3, 4, 5) => 5,
    (3, 4, 6) => 5,
)
# job-i operation-l pairs
JO = [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (3, 1), (3, 2), (3, 3), (3, 4)];
# JobLMachine: from (i, l) job-i operation-l pairs to the machines that can be potentially used
# JobLMachine1: if it can only use 1 machine
# JobLMachine2: if it can choose between 2 machines
JobLMachine = Dict(
    (1, 1) => [1],
    (1, 2) => [2],
    (1, 3) => [3], 
    (1, 4) => [5, 6], 
    (1, 5) => [5, 6], 
    (2, 1) => [1],
    (2, 2) => [2],
    (2, 3) => [3], 
    (2, 4) => [4], 
    (2, 5) => [5, 6], 
    (2, 6) => [5, 6], 
    (3, 1) => [1],
    (3, 2) => [6],
    (3, 3) => [6], 
    (3, 4) => [5, 6]
)
JobLMachine1 = Dict(1 => [1, 2, 3], 2 => [1, 2, 3, 4], 3 => [1, 6])
JobLMachine2 = Dict(1 => [5, 6], 2 => [5, 6], 3 => [])
;

In [ ]:
using Gurobi, JuMP
# Initialize the model with the Gurobi solver
model = Model(Gurobi.Optimizer)
validx = [ (i, l, j, k) for (i, l) in JO for k in JobLMachine[(i, l)] for j in 1:nk[k] ]

In [2]:
x = []

# Decision Variables
validx = [ (i, l, j, k) for (i, l) in JO for k in JobLMachine[(i, l)] for j in 1:nk[k] ]
@variable(model, x[validx], Bin)
@variable(model, h[[ (j, k) for k in 1:MM for j in 1:nk[k] ]] >= 0)
@variable(model, s[[ (j, k) for k in 1:MM for j in 1:nk[k] ]] >= 0)
@variable(model, s0[ k in 1:MM ] >= 0)
@variable(model, TX[[ (j, k) for k in 1:MM for j in 1:nk[k] ]] >= 0)

valid_deltas = []
for (group, machines) in MG  # now group is (i, g) pair, machines are idx of M that (i, g) might use
    for a in 1:length(machines)
        for b in (a+1):length(machines)  # Ensure b > a to get unique pairs without repetition
            for ll in GroupOps[(group)], lll in GroupOps[(group)]
                if ll != lll
                    for jj in 1:nk[machines[a]], jjj in 1:nk[machines[b]]

                    
                end




            
            
                push!( valid_deltas, (group[1], group[2], machines[a], machines[b], jj, jjj) )
            end
        end
    end
end
@variable(model, delta[ valid_deltas ] >= 0);
@variable(model, h_star >= 0);




@variable(model, x[i=1:num_jobs, l=1:num_operations, j=1:num_machines, k=1:num_machines], Bin)
@variable(model, h[j=1:num_machines, k=1:num_machines] >= 0)
@variable(model, s[j=1:num_machines, k=1:num_machines] >= 0)
@variable(model, s0[k=1:num_machines] >= 0)
@variable(model, h_star >= 0)

# Known Variables (parameters)
n = ... # max number of order-positions per machine, dictionary keyed by machine
N = ... # job & operation pairs that can possibly use machine k, dictionary keyed by machine
t = ... # time needed for each operation on each machine, 3D array or dictionary
cS = ... # total numbers of time each operation will use any machine in S, 2D array or dictionary

# Objective: Minimize the maximum completion time
@objective(model, Min, h_star)

# Constraint Set 1: Each operation is processed a specific number of times
@constraint(model, c1[i=1:num_jobs, l=1:num_operations], sum(x[i,l,j,k] for k=1:num_machines for j=1:n[k]) == cS[i,l])

# Constraint Set 2: At most one operation at each order position on each machine
@constraint(model, c2[j=1:num_machines, k=1:num_machines], sum(x[i,l,j,k] for (i, l) in N[k]) <= 1)

# Constraint Set 3: Define the processing time for each order position on each machine
@constraint(model, c3[j=1:num_machines, k=1:num_machines], sum(t[i,l,k] * x[i,l,j,k] for (i, l) in N[k]) == s[j,k])

# Define h variables
@constraint(model, h[1,k] == s0[k] for k in 1:num_machines)
@constraint(model, h_constraints[j=2:n[k], k=1:num_machines], h[j,k] == sum(s[q,k] for q in 1:j-1))

# Constraint Set 4: Precedence constraints
M = 1e6 # A large constant
@constraint(model, c4[i=1:num_jobs, g=1:num_operations], begin
    for a in machine_groups[g]
        for b in machine_groups[g+1]
            for l' in 1:num_operations
                for l'' in 1:num_operations
                    for j' in 1:n[a]
                        for j'' in 1:n[b]
                            h[j',a] + t[i,l',a] * x[i,l',j',a] <= h[j'',b] + M * (1 - x[i,l',j',a]) + M * (1 - x[i,l'',j'',b])
                        end
                    end
                end
            end
        end
    end
end)

# Constraint Set 5: No overlap within the same machine group
@constraint(model, c5[i=1:num_jobs, g=1:num_operations], begin
    delta = ... # A binary variable or a parameter that needs to be defined
    for a in machine_groups[g]
        for b in machine_groups[g]
            for (l', l'') in [(l', l'') for l' in 1:num_operations, l'' in 1:num_operations if l' != l'']
                for j' in 1:n[a]
                    for j'' in 1:n[b] if j' != j''
                        # Ensuring that one operation finishes before the next one starts within the same machine group
                        h[j',a] + t[i,l',a] * x[i,l',j',a] <= h[j'',b] + M * (1 - x[i,l',j',a]) + M * (1 - x[i,l'',j'',b]) + delta * M
                        h[j'',b] + t[i,l'',b] * x[i,l'',j'',b] <= h[j',a] + M * (1 - x[i,l',j',a]) + M * (1 - x[i,l'',j'',b]) + (1 - delta) * M
                    end
                end
            end
        end
    end
end)

# Constraint Set 6: h_star is greater than or equal to the end time of the last operation on each machine
@constraint(model, c6[k=1:num_machines], h_star >= h[n[k],k] + s[n[k],k])

# Optimize the model
optimize!(model)

# Output the solution
println("Objective Value: ", objective_value(model))
for k in 1:num_machines

Dict{Tuple{Int64, Int64}, Vector{Int64}} with 1 entry:
  (1, 1) => [1, 2, 3]

In [9]:
valid_combinations = [ (i, l, j, k) for (i, l) in JO for k in MM for j in 1:nk[k] ]
# Decision Variables - Sparse definition using a dictionary
@variable(model, x[valid_combinations], Bin)

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, [(1, 1, 1, 6), (1, 1, 2, 6), (1, 1, 3, 6), (1, 1, 4, 6), (1, 1, 5, 6), (1, 1, 6, 6), (1, 1, 7, 6), (1, 2, 1, 6), (1, 2, 2, 6), (1, 2, 3, 6)  …  (3, 3, 5, 6), (3, 3, 6, 6), (3, 3, 7, 6), (3, 4, 1, 6), (3, 4, 2, 6), (3, 4, 3, 6), (3, 4, 4, 6), (3, 4, 5, 6), (3, 4, 6, 6), (3, 4, 7, 6)]
And data, a 105-element Vector{VariableRef}:
 x[(1, 1, 1, 6)]
 x[(1, 1, 2, 6)]
 x[(1, 1, 3, 6)]
 x[(1, 1, 4, 6)]
 x[(1, 1, 5, 6)]
 x[(1, 1, 6, 6)]
 x[(1, 1, 7, 6)]
 x[(1, 2, 1, 6)]
 x[(1, 2, 2, 6)]
 x[(1, 2, 3, 6)]
 x[(1, 2, 4, 6)]
 x[(1, 2, 5, 6)]
 x[(1, 2, 6, 6)]
 ⋮
 x[(3, 3, 3, 6)]
 x[(3, 3, 4, 6)]
 x[(3, 3, 5, 6)]
 x[(3, 3, 6, 6)]
 x[(3, 3, 7, 6)]
 x[(3, 4, 1, 6)]
 x[(3, 4, 2, 6)]
 x[(3, 4, 3, 6)]
 x[(3, 4, 4, 6)]
 x[(3, 4, 5, 6)]
 x[(3, 4, 6, 6)]
 x[(3, 4, 7, 6)]